# Azure AI Agent Service
Service will combine knowledge and actions with AI Agents. 

# Travel Example
I am planning a trip to Atlanta, based on the weather, plan a wardrobe for me

Solution may use Weather, Clothing and Transportation Agents to come up with a list of actions to take. May mean extended time to get from one loction to another. May mean requesting a rideshare service ahead. 

# How is this different than OpenAI Assistants API?
- The Assistants API has Assistants (Agents), Tools, Threads and Messages
- The OpenAI Assistants API has been in Preview for quite awhile with no plan to GA anytime soon
- Customers need a service like this for even just simple things like chat history storage
- The service is tied to an endpoint and doesn't easily provide Highly Available solution or implement Disaster Recovery

# How will Azure AI Agent Service Work
- This service will provide a robust set of AI searvies all within an Agent
- With a drop in option for current users of the OpenAI Assistants API, this will be a enterprise ready solution that will go GA
- [Others](https://learn.microsoft.com/en-us/azure/ai-services/agents/overview)
- [Agent vs Assistants API](https://learn.microsoft.com/en-us/azure/ai-services/agents/overview#comparing-azure-agents-and-azure-openai-assistants)

# So what who cares?
Customers want enterprise ready agent services and don't want to maintain all the infrastructure while have state of the art secuity and robust Responsible AI built in

# Getting Started
- [Quickstart in C# and Python](https://learn.microsoft.com/en-us/azure/ai-services/agents/quickstart?pivots=programming-language-python-azure)

## Install a Standard Project
- Use the template in the Quickstart
- [Basic or Standard](https://learn.microsoft.com/en-us/azure/ai-services/agents/quickstart?pivots=programming-language-python-azure#choose-basic-or-standard-agent-setup)
    - Basic: This would be like you have today with the Assistants API
    - Standard: Adds more services like AI Search and Storage to work securely

## Modules Needed
```bash
pip install azure-ai-projects
pip install azure-identity
```

## Samples and More
https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/agents

In [6]:
# Load modules
import os
import sys
import json, jsonref
import requests
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    BingGroundingTool,
    AzureAISearchTool,
    ConnectionType
)
from azure.ai.inference.models import SystemMessage, UserMessage

from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential
from typing import Any
from pathlib import Path
import time
import logging
from dotenv import load_dotenv

_ = load_dotenv()

In [4]:
# Helper functions
def printMessages(data):
    # Loop through the messages in reverse order
    for message in reversed(data):
        print(f"{message["role"]}: {message["content"][0]["text"]["value"]}")    
        if 'annotations' in message["content"][0]["text"]:
            for annotation in message["content"][0]["text"]["annotations"]:
                print(f"Annotation: {annotation}")

# Grounding with Bing
Even though this service might 'appear' to work when you go through the [quickstart](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=code-example), it might not actually be working. Make sure to include a question that is from a recent event. 

Make sue to follow the steps to create the Bing Grounding resource first

Like many Viking fans, just ask about their last game

In [5]:

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What was the result of the last Vikings game?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Retrieve run step details to get Bing Search query link
    # To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    run_steps_data = run_steps['data']
    print(f"Last run step detail: {run_steps_data}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    printMessages(messages['data'])

Created agent, ID: asst_7kF6LIDCHkd6alpzxP6VAPlh
Created thread, ID: thread_E9v6aFEXXZai1OJd3U25Ujfq
Created message, ID: msg_s1TEdKrJNbBfs2vG1A1GJwAS
Run finished with status: RunStatus.COMPLETED
Last run step detail: [{'id': 'step_YhohOwoDN2AJKfEMwUiPZE7W', 'object': 'thread.run.step', 'created_at': 1737990163, 'run_id': 'run_zicjCj4Xa8AUXUTY3jjNduy4', 'assistant_id': 'asst_7kF6LIDCHkd6alpzxP6VAPlh', 'thread_id': 'thread_E9v6aFEXXZai1OJd3U25Ujfq', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1737990164, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_5ma6MQXJ8zOGDJ2dGDRhx5Aw'}}, 'usage': {'prompt_tokens': 2390, 'completion_tokens': 37, 'total_tokens': 2427}}, {'id': 'step_yVj8s9UYV2l3tgS3EWRHAbdD', 'object': 'thread.run.step', 'created_at': 1737990161, 'run_id': 'run_zicjCj4Xa8AUXUTY3jjNduy4', 'assistant_id': 'asst_7kF6LIDCHkd6alpzxP6VAPlh', 'thr

# AI Search
Follow the configuration and setup needed on the [Overview](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search) tab first.

https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search

https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/agents/sample_agents_azure_ai_search.py

Azure AI Search indexes must meet the following requirements:

- The index must contain at least one searchable & retrievable text field (type Edm.String)
- The index must contain at least one searchable vector field (type Collection(Edm.Single))
- The index must use a vector profile/integrated vectorization

Create an Index to use:
- Upload a sample PDF or more to a storage account contains
    - [Sample: Northwind Standard Benefits Detail (PDF)](https://github.com/Azure-Samples/azure-search-openai-demo/blob/main/data/Northwind_Standard_Benefits_Details.pdf?raw=true)
- Grant the Managed Identity of the AI Search Service `Storage Blob Container Reader` at the Resource level and also the Container
- Use the "Import" option from the AI Search Service to import the files from the Blob
- Name the index `vector-victor`

> NOTE: Even if you don't have a valid index, this still "tries" to answer the question. That is a bug and will hopefully be fixed soon


In [7]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# [START create_agent_with_azure_ai_search_tool]
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:
    if conn.connection_type == ConnectionType.AZURE_AI_SEARCH:
        conn_id = conn.id
        break

print(conn_id)

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name="vector-victor")

# Create agent with AI search tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model='gpt-4o-mini',
        name="benefits-assistant",
        instructions="You are a helpful assistant",
        tools=ai_search.definitions,
        tool_resources=ai_search.resources,
        headers={"x-ms-enable-preview": "true"},
    )
    # [END create_agent_with_azure_ai_search_tool]
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Which is more comprehensive, Northwind Health Plus vs Northwind Standard?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    printMessages(messages.data)

/subscriptions/4133db51-7a0f-46c5-b97d-fc06d648f32b/resourceGroups/rg-dai-usw/providers/Microsoft.MachineLearningServices/workspaces/proj-dai-usw/connections/srchdaiuse
Created agent, ID: asst_xYb8KwkKlQyFZJWGSt5ob3nD
Created thread, ID: thread_csrHDykkU2BgjPiTxJKKT3fb
Created message, ID: msg_3IZPNI0S5UgyZLPpAqe9a3vR
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_qLchGbR746xCBocJBVpGvqZn', 'object': 'thread.message', 'created_at': 1737990752, 'assistant_id': 'asst_xYb8KwkKlQyFZJWGSt5ob3nD', 'thread_id': 'thread_csrHDykkU2BgjPiTxJKKT3fb', 'run_id': 'run_1VgAzZHE6wQfyJRB5xq4xvpz', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Northwind Health Plus is more comprehensive than Northwind Standard. Here are the key differences:\n\n1. **Emergency Services**: Northwind Health Plus covers emergency services, whereas Northwind Standard does not.\n2. **Mental Health and Substance Abuse**: Northwind Health Plus i